#### Controlling table inheritance with mixins

The `__tablename__` attribute may be used to _provide a function that will determine the name of the table_ used for each class in an __inheritance hierarchy__, as well as whether a class has its _own distinct table_.

This is achieved using the `declared_attr` indicator __in conjunction with a method__ named `__tablename__()`. _Declarative_ will __always invoke__ `declared_attr` for the special names `__tablename__`, `__mapper_args__` and `__table_args__` function _for each mapped class in the hierarchy_, __except if overridden in a subclass__. The function therefore needs to _expect to receive_ __each class individually__ and to _provide_ the __correct answer for each__.

For example, to _create a mixin_ that gives every class a __simple table name based on class name__.

In [1]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import (
    declared_attr, declarative_mixin, declarative_base, has_inherited_table
)

In [2]:
Base = declarative_base()

In [3]:
@declarative_mixin
class TablenameMixin:
    @declared_attr
    def __tablename__(cls):
        return cls.__name__.lower()

In [4]:
class Person(TablenameMixin, Base):
    id = Column(Integer, primary_key=True)
    discriminator = Column("type", String(50))
    
    __mapper_args__ = {"polymorphic_on": discriminator}

In [5]:
class Engineer(Person):
    __tablename__ = None
    __mapper_args__ = {"polymorphic_identity": "engineer"}
    primary_language = Column(String(50))

Alternatively, we can modify our `__tablename__` function to return `None` for _subclasses_, using `has_inherited_table()`. This has the _effect of those subclasses_ __being mapped with single table inheritance against the parent__.

In [6]:
@declarative_mixin
class InheritedTablenameMixin:
    @declared_attr
    def __tablename__(cls):
        if has_inherited_table(cls):
            return None
        return cls.__name__.lower()

In [7]:
class PersonParent(InheritedTablenameMixin, Base):
    id = Column(Integer, primary_key=True)
    discriminator = Column("type", String(50))
    
    __mapper_args__ = {"polymorphic_on": discriminator}

In [8]:
class EngineerInheritor(PersonParent):
    primary_language = Column(String(50))
    __mapper_args__ = {"polymorphic_identity": "engineerinheritor"}